In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import csv
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import seaborn as sns

from lieroy import se3
from recova.clustering import CenteredClusteringAlgorithm, RegistrationPairClusteringAdapter
from recova.covariance import SamplingCovarianceComputationAlgorithm
from recova.registration_result_database import RegistrationPairDatabase

In [3]:
db = RegistrationPairDatabase('/home/dlandry/dataset/db_eth_05/')
pairs = db.registration_pairs()

In [4]:
clustering_algo = CenteredClusteringAlgorithm(radius=0.005, k=20, n_seed_init=20)
clustering_algo.seed_selector = 'localized'
clustering = RegistrationPairClusteringAdapter(clustering_algo)

covariance_algo = SamplingCovarianceComputationAlgorithm(clustering_algorithm=clustering)

In [12]:

def distance_of_one_pair(pair):
    ground_truth = pair.ground_truth()
    cluster = clustering.compute(pair)
    
    group_results = pair.registration_results()
    indiv_distances = np.empty(len(cluster))
    for j in range(len(cluster)):
        delta = np.linalg.inv(ground_truth) @ group_results[cluster[j]]
        indiv_distances[j] = np.linalg.norm(se3.log(delta))
    
    mean, covaiance = se3.gaussian_from_sample(group_results[cluster])
    
    delta = se3.log(np.linalg.inv(ground_truth) @ mean)
    distance = np.linalg.norm(delta)
    min_distance = np.min(indiv_distances)
    print('{}: {}'.format(pair, distance))
    print('Min dist: {}'.format(min_distance))
    return distance, min_distance

In [18]:
distance_of_one_pair(pairs[5])

Registration Pair: apartment-0003-0002: 0.0049440420058525164
Min dist: 0.004684889791613866


(0.0049440420058525164, 0.004684889791613866)

### 

#results = [distance_of_one_pair(x) for x in pairs]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.map(distance_of_one_pair, pairs)
    

In [ ]:
covariances = [covariance_algo.compute(x) for x in pairs]

In [ ]:
distances

In [ ]:
with open('/home/dlandry/distances_of_mean.csv', 'w') as f:
    writer = csv.DictWriter(f, ['location', 'reading', 'reference', 'distance', 'n_samples'])
    writer.writeheader()
    for i, pair in enumerate(pairs):
        writer.writerow({
            'location': pair.dataset,
            'reading': pair.reading,
            'reference': pair.reference,
            'distance': distances[i],
            'n_samples': 
        })

In [ ]:
sns.distplot(distances, rug=True)
plt.xlim([0.0, 0.2])
plt.show()

In [ ]:
distances, closest = zip(*results)

In [ ]:
distance, closest = np.array(distances), np.array(closest)

In [ ]:
distance - closest

In [ ]:
np.max(distance - closest)

In [ ]:
distance - closest

In [ ]:
sns.distplot(closest, rug=True)
plt.xlim([0.0, 0.3])
plt.show()